### 使用说明
1. 输入所需信息：
    * working file的文件名(带后缀)，如```PwQ_Pre-Allocation W34_Planners_WorkingFile - Copy.xlsm```，以及sheet_name如```2920```
    * excel属于你工作内容的起始行，如PIC为Yanqing的工作内容从excel中的第```8```行开始
    * 用于计算温饱线的excel文件名称(带后缀)，如```AGL needs W34.xlsx```，以及sheet_name如```Stock needed```
2. 运行本文件，在 [Cell] 中点击 Run All，会自动生成两个文件：
    * ```_pre_allocation_res.xlsx``` 储存计算结果
    * ```_pre_allocation_log.log```储存计算日志以信息查询与比对


In [1]:
# 注：使用前请先确保以下信息填写正确：
working_file_name = 'PwQ_Pre-Allocation W38_Planners_WorkingFile - YQ.xlsm'
working_file_sheet_name = '2920'
start_row = 8
agl_needs_file_name = 'AGL needs W38.xlsx'
agl_needs_file_sheet_name = 'Stock needed'
pic = 'Yanqing'

In [23]:
import pandas as pd
import numpy as np
from pre_allocation import Solution
import logging
import xlsxwriter
from datetime import date
from openpyxl import load_workbook
from openpyxl.formula.translate import Translator

In [3]:
Log_Format = "%(levelname)s %(asctime)s - %(message)s"
today = date.today()
today = today.strftime("%Y-%m-%d")
file_name = today + '_pre_allocation'
logging.basicConfig(filename = str(today) + "_pre_allocation_log.log",
                    encoding = "utf-8",
                    filemode = "w",
                    format = Log_Format, 
                    level = logging.INFO)
logger = logging.getLogger()

In [4]:
working_file = pd.read_excel(working_file_name, sheet_name=working_file_sheet_name).iloc[5:, 1:]
working_file.columns = working_file.iloc[0]
working_file = working_file[working_file['PIC'] == pic]
working_file.index = np.arange(start_row, len(working_file) + start_row)
columns = ['Material', 'PIC', 'Prepack', 'SOH', 'Avg Ind.Req (M)','KR(1200) -', 'Direct KR', 'HK(1220) -', 'Direct HK','AU(1180) -', 'Direct AU', 'TW(1280) -', 'Direct TW', 'CN(1190) -',
       'Direct CN', 'JP(1230) -', 'Direct JP', 'TH(1340) -', 'Direct TH']
working_file = working_file[columns].replace(np.nan, 0)
working_file = working_file.astype({'Avg Ind.Req (M)':int})

In [5]:
working_file.head(20)

5,Material,PIC,Prepack,SOH,Avg Ind.Req (M),KR(1200) -,Direct KR,HK(1220) -,Direct HK,AU(1180) -,Direct AU,TW(1280) -,Direct TW,CN(1190) -,Direct CN,JP(1230) -,Direct JP,TH(1340) -,Direct TH
8,C103300030,Yanqing,480,0,0,0,0,0,0,0,0,0,0,0,0,0,0,300,0
9,C105000010,Yanqing,2000,11194,2058,0,0,0,0,0,0,0,0,26000,26000,20000,0,0,0
10,C105600001,Yanqing,630,7840,139,0,0,0,0,0,0,0,0,0,0,17640,0,0,0
11,C106800001,Yanqing,480,622,142,0,0,0,0,0,0,0,0,0,0,0,0,300,0
12,C107373100,Yanqing,120,4737,100,0,0,240,0,0,0,0,0,0,0,0,0,0,0
13,C110300010,Yanqing,350,11476,172,2100,0,0,0,0,0,0,0,0,0,7100,0,0,0
14,C110300020,Yanqing,350,164,155,0,0,0,0,0,0,0,0,0,0,7210,0,0,0
15,C110571001,Yanqing,2000,2000,0,0,0,0,0,0,0,0,0,0,0,22205,0,0,0
16,C111000999,Yanqing,120,9623,1317,4200,0,0,0,0,0,0,0,0,0,0,0,0,0
17,C111100999,Yanqing,100,0,27,0,0,0,0,0,0,0,0,0,0,90,0,0,0


In [6]:
newdf = working_file
newdf['KR(1200) -'] = np.where(newdf['KR(1200) -'] - newdf['Direct KR'] <= 0, 0, newdf['KR(1200) -'] - newdf['Direct KR'])
newdf['HK(1220) -'] = np.where(newdf['HK(1220) -'] - newdf['Direct HK'] <= 0, 0, newdf['HK(1220) -'] - newdf['Direct HK'])
newdf['AU(1180) -'] = np.where(newdf['AU(1180) -'] - newdf['Direct AU'] <= 0, 0, newdf['AU(1180) -'] - newdf['Direct AU'])
newdf['TW(1280) -'] = np.where(newdf['TW(1280) -'] - newdf['Direct TW'] <= 0, 0, newdf['TW(1280) -'] - newdf['Direct TW'])
newdf['CN(1190) -'] = np.where(newdf['CN(1190) -'] - newdf['Direct CN'] <= 0, 0, newdf['CN(1190) -'] - newdf['Direct CN'])
newdf['JP(1230) -'] = np.where(newdf['JP(1230) -'] - newdf['Direct JP'] <= 0, 0, newdf['JP(1230) -'] - newdf['Direct JP'])
newdf['TH(1340) -'] = np.where(newdf['TH(1340) -'] - newdf['Direct TH'] <= 0, 0, newdf['TH(1340) -'] - newdf['Direct TH'])
newdf['after DD'] = np.where(newdf['SOH'] - newdf['Avg Ind.Req (M)'] <= 0, 0, newdf['SOH'] - newdf['Avg Ind.Req (M)'])
newdf.drop(['Direct KR','Direct HK','Direct AU','Direct TW','Direct CN','Direct JP','Direct TH'], axis=1,inplace=True)

In [7]:
newdf.head(20)

5,Material,PIC,Prepack,SOH,Avg Ind.Req (M),KR(1200) -,HK(1220) -,AU(1180) -,TW(1280) -,CN(1190) -,JP(1230) -,TH(1340) -,after DD
8,C103300030,Yanqing,480,0,0,0,0,0,0,0,0,300,0
9,C105000010,Yanqing,2000,11194,2058,0,0,0,0,0,20000,0,9136
10,C105600001,Yanqing,630,7840,139,0,0,0,0,0,17640,0,7701
11,C106800001,Yanqing,480,622,142,0,0,0,0,0,0,300,480
12,C107373100,Yanqing,120,4737,100,0,240,0,0,0,0,0,4637
13,C110300010,Yanqing,350,11476,172,2100,0,0,0,0,7100,0,11304
14,C110300020,Yanqing,350,164,155,0,0,0,0,0,7210,0,9
15,C110571001,Yanqing,2000,2000,0,0,0,0,0,0,22205,0,2000
16,C111000999,Yanqing,120,9623,1317,4200,0,0,0,0,0,0,8306
17,C111100999,Yanqing,100,0,27,0,0,0,0,0,90,0,0


In [7]:
skuList = list(working_file['Material'])
workbook = load_workbook(filename=agl_needs_file_name)
worksheet = workbook[agl_needs_file_sheet_name]
formula_row = 3
for sku in skuList:
    for col in ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
        cell = col + str(formula_row)
        if col == 'A':
            worksheet[cell] = sku
        else:
            formula = worksheet[col+'3'].value
            worksheet[cell] = Translator(formula, origin=col+'3').translate_formula(cell)
    formula_row += 1 
workbook.save("agl_needs_updated.xlsx")

In [9]:
# 神奇bug，openpyxl保存的excel并未真正保存 -> https://codeantenna.com/a/iRcZiE5TVp
# 需要用win32com模拟打开并关闭excel来完成一次储存操作
# https://bbs.huaweicloud.com/blogs/363106 -> win32com安装，仅限windows
# pip install pypiwin32 或 pip install pywin32
# !!!以下命令仅用于windows系统，mac只能手动打开excel保存后关闭，然后跳过该代码继续执行

# 如需运行以下代码，取消注释即可

from win32com.client import Dispatch
def open_save(filename):
    xlApp = Dispatch('Excel.Application')
    xlApp.Visible = False
    xlBook = xlApp.Workbooks.Open(filename)
    xlBook.Save()
    xlBook.Close()
open_save(r"C:\Users\yawang\Desktop\supply_planning_trivial_matters\pre_allocation\agl_needs_updated.xlsx")

In [28]:
agl_needs = pd.read_excel("agl_needs_updated.xlsx", sheet_name=agl_needs_file_sheet_name).iloc[1:len(skuList)+2].fillna(0).rename({'SKU':'Material'}, axis=1)
agl_needs.drop(['Designation'], axis=1, inplace=True)
agl_needs = agl_needs.astype({'KR':int, 'HK': int, 'AU': int, 'TW':int, 'CN': int, 'JP': int, 'TH': int})
print(len(agl_needs))
agl_needs.head(20)

299


,Material,KR,HK,AU,TW,CN,JP,TH
1,C103300030,0,0,0,0,0,0,-200
2,C105000010,0,0,0,0,-4435,-5584,0
3,C105600001,0,0,0,0,0,4314,0
4,C106800001,0,0,0,0,0,0,-200
5,C107373100,0,-124,0,0,0,0,0
6,C110300010,599,0,0,0,0,2215,0
7,C110300020,0,0,0,0,0,-1934,0
8,C110571001,0,0,0,0,0,221328,0
9,C111000999,4082,0,0,0,0,0,0
10,C111100999,0,0,0,0,0,9,0


In [24]:
df = pd.merge(newdf, agl_needs, on='Material', how='outer')
df.drop(['SOH','Avg Ind.Req (M)'], axis=1, inplace=True)
df.insert(3, 'after DD', df.pop('after DD'))
df.index = np.arange(start_row, len(df) + start_row)
df.head(20)

,Material,PIC,Prepack,after DD,KR(1200) -,HK(1220) -,AU(1180) -,TW(1280) -,CN(1190) -,JP(1230) -,TH(1340) -,KR,HK,AU,TW,CN,JP,TH
8,C103300030,Yanqing,480,0,0,0,0,0,0,0,300,0,0,0,0,0,0,-200
9,C105000010,Yanqing,2000,9136,0,0,0,0,0,20000,0,0,0,0,0,-4435,-5584,0
10,C105600001,Yanqing,630,7701,0,0,0,0,0,17640,0,0,0,0,0,0,4314,0
11,C106800001,Yanqing,480,480,0,0,0,0,0,0,300,0,0,0,0,0,0,-200
12,C107373100,Yanqing,120,4637,0,240,0,0,0,0,0,0,-124,0,0,0,0,0
13,C110300010,Yanqing,350,11304,2100,0,0,0,0,7100,0,599,0,0,0,0,2215,0
14,C110300020,Yanqing,350,9,0,0,0,0,0,7210,0,0,0,0,0,0,-1934,0
15,C110571001,Yanqing,2000,2000,0,0,0,0,0,22205,0,0,0,0,0,0,221328,0
16,C111000999,Yanqing,120,8306,4200,0,0,0,0,0,0,4082,0,0,0,0,0,0
17,C111100999,Yanqing,100,0,0,0,0,0,0,90,0,0,0,0,0,0,9,0


In [25]:
def getFinalResult(row):
    prepack = row['Prepack']
    own = row['after DD']
    needs = [row['KR'], row['HK'], row['AU'], row['TW'], row['CN'], row['JP'], row['TH']]
    wants = [row['KR(1200) -'], row['HK(1220) -'], row['AU(1180) -'], row['TW(1280) -'], row['CN(1190) -'], row['JP(1230) -'], row['TH(1340) -']]
    s = Solution(needs, wants, prepack, own)
    res = s.getDistribution()
    logger.info(f"{row['Material']} ==> \n{s.explanation}\n")
    newCols = ['KR(1200) +', 'HK(1220) +', 'AU(1180) +', 'TW(1280) +', 'CN(1190) +', 'JP(1230) +', 'TH(1340) +', 'comments']
    for i in range(len(newCols)):
        if i == len(newCols) - 1:
            if s.ok:
                row[newCols[i]] = 'ok'
            else: 
                row[newCols[i]] = ''
        else:
            row[newCols[i]] = int(res[i])

    return row

In [26]:
res = df.apply(getFinalResult, axis=1)

In [29]:
res.head(20)

,Material,PIC,Prepack,after DD,KR(1200) -,HK(1220) -,AU(1180) -,TW(1280) -,CN(1190) -,JP(1230) -,...,JP,TH,KR(1200) +,HK(1220) +,AU(1180) +,TW(1280) +,CN(1190) +,JP(1230) +,TH(1340) +,comments
8,C103300030,Yanqing,480,0,0,0,0,0,0,0,...,0,-200,0,0,0,0,0,0,0,
9,C105000010,Yanqing,2000,9136,0,0,0,0,0,20000,...,-5584,0,0,0,0,0,0,8000,0,
10,C105600001,Yanqing,630,7701,0,0,0,0,0,17640,...,4314,0,0,0,0,0,0,7560,0,
11,C106800001,Yanqing,480,480,0,0,0,0,0,0,...,0,-200,0,0,0,0,0,0,300,ok
12,C107373100,Yanqing,120,4637,0,240,0,0,0,0,...,0,0,0,240,0,0,0,0,0,ok
13,C110300010,Yanqing,350,11304,2100,0,0,0,0,7100,...,2215,0,2100,0,0,0,0,7100,0,ok
14,C110300020,Yanqing,350,9,0,0,0,0,0,7210,...,-1934,0,0,0,0,0,0,0,0,
15,C110571001,Yanqing,2000,2000,0,0,0,0,0,22205,...,221328,0,0,0,0,0,0,2000,0,
16,C111000999,Yanqing,120,8306,4200,0,0,0,0,0,...,0,0,4200,0,0,0,0,0,0,ok
17,C111100999,Yanqing,100,0,0,0,0,0,0,90,...,9,0,0,0,0,0,0,0,0,


In [30]:
writer = pd.ExcelWriter('pre_allocation_res.xlsx', engine='xlsxwriter')
res.to_excel(writer, sheet_name='Sheet1')
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

bottom_row = str(len(res) + 1)
format_wants = ['#FFC7CE', 'F2:L' + bottom_row]
format_needs = ['#CCFFFF', 'M2:S' + bottom_row]
format_res = ['#E5FFCC', 'T2:Z' + bottom_row]
format_list = [format_wants, format_needs, format_res] # 3个涂色，wants, needs, res, 每个format信息为[color, left_top:right_bottom]
for each_format in format_list:
    fm = workbook.add_format({'bg_color': each_format[0]})
    area = each_format[1]
    worksheet.conditional_format(area, {'type': 'no_blanks','format': fm})
worksheet.freeze_panes(1,5)
writer.save()

In [ ]:
# 似乎是windows特有的问题，如果不shutdown，无法重写或删除log日志
logging.shutdown()

#### 以下代码仅用于单测

In [ ]:
prepack = 69
own = 47
needs = [0,0,-13,-37,-765,0,0]
wants = [0,0,16,0,828, 0, 0, 0]
s = Solution(needs, wants, prepack, own)
print(s.getDistribution())
print(s.explanation)
print(s.ok)